In [1]:
from lxml import html
import requests
import unicodecsv as csv
import argparse
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

from re import sub
from decimal import Decimal


address=[]
price=[]
detail=[]
link=[]
table1=[]
table2=[]

In [2]:
beds=[]
baths=[]
links=[]
price=[]
sq_feet=[]

cities= ['Rexburg_ID','Boise_ID']
city_select=[]
pages=[1,2]


req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
    for city in cities:
        for page in pages:
        
            url = 'https://www.realtor.com/realestateandhomes-search/'+str(city)+'/pg-'+str(page)

            r = s.get(url, headers=req_headers)

            soup = BeautifulSoup(r.content, 'html.parser')

            hrefs = soup.find_all('div', class_ = 'jsx-11645185 card-box type-srp-result')
            for link in hrefs:
                links.append(link.a['href'])


            summary=soup.find_all('div', class_ = 'jsx-11645185 property-wrap')


            p=soup.find_all(class_=re.compile("rui__x3geed-0 kitA-dS"))
            skip='price-reduced-amount'
            for i in p:
                if skip in str(i):
                    pass
                else:
                    price.append(i.text)
                    city_select.append(city)
    
            

        
                
                            



In [3]:
df=pd.DataFrame()
df['price'] = price
df['link'] = links
df['city']=city_select

df

,price,link,city
0,"$465,000",/realestateandhomes-detail/4983-S-2000-W_Rexbu...,Rexburg_ID
1,"$499,000",/realestateandhomes-detail/3029-N-Salem-Rd_Rex...,Rexburg_ID
2,"$253,000",/realestateandhomes-detail/271-W-1st-N_Rexburg...,Rexburg_ID
3,"$595,000",/realestateandhomes-detail/253-E-5th-S_Rexburg...,Rexburg_ID
4,"$375,000",/realestateandhomes-detail/6344-S-3100-W_Rexbu...,Rexburg_ID
...,...,...,...
163,"$629,990",/realestateandhomes-detail/8333-S-Snow-Bird-Av...,Boise_ID
164,"$725,000",/realestateandhomes-detail/9641-W-Sultana-Dr_G...,Boise_ID
165,"$925,000",/realestateandhomes-detail/2926-S-Barnside-Way...,Boise_ID
166,"$619,990",/realestateandhomes-detail/8351-S-Snow-Bird-Av...,Boise_ID


In [4]:
#delays=[7,4,6,15,10,19]
#year_built=[0]*len(df)
year_built=[]
span_total=[]
link_total=[]
#price_history=[]
df['sold_data'] = ''

with requests.Session() as s:
    for i in range(0,len(df)):
        row=df.loc[i]
        url = row['link']
    
        price_history=[]
        
            #delay=np.random.choice(15)
        
        r = s.get("https://www.realtor.com/"+url, headers=req_headers)
        soups = BeautifulSoup(r.content, 'lxml')
        span = soups.find('body').find_all("ul")
        
        span_total.append(span)
        link_total.append("https://www.realtor.com/"+url)
        ph=soups.find_all('div',class_="listing-subsection listing-subsection-price")
        price_history.append(ph)
        
        sold_prices=[]
      
        
        for p in price_history:
            for t in p:
                x=t.find_all('tr')
                for o in x:
                    
                    sell=o.get_text()

                    if 'Listed' in sell:
                        sell = sell.split("\n")
                        d={
                            'date':sell[1],
                            'sold_price':sell[3]
                        }
                        sold_prices.append(d)
     
        row['sold_data'] = sold_prices
        
        
        
           

In [5]:
year_b=[0]*len(df)
i=0
for sp in span_total:
    for ul in sp:
        for li in ul:
            lines=li.get_text()
            if 'Land/Lot' in lines:
                year_b[i]=(lines)
            if 'Built' in lines:
                #print(lines)
                #input("")
                if len(lines) < 15:
                    year_b[i]=(lines)

    i+=1

                        

In [6]:
price_sq_feet=[0]*len(df)
bed=[0]*len(df)
bath=[0]*len(df)
sqft=[0]*len(df)
source=[0]*len(df)
House_Style=[0]*len(df)
County=[0]*len(df)
i=0

for sp in span_total:
    for ul in sp:
        for li in ul:
            lines=li.get_text()  
            if 'Price/Sq Ft' in lines:
                price_sq_feet[i]=(lines)
                
            if 'Bedrooms:' in lines:
                bed[i]=(lines)
                
            if 'Total Bathrooms:' in lines:
                bath[i] = lines
            if 'sq ft' in lines:
                sqft[i] = lines
            if 'Source Property Type:' in lines:
                source[i] = lines
            if 'House Style:' in lines:
                House_Style[i] = lines
            if 'Architectural Style' in lines:
                House_Style[i] = lines
            if 'County:' in lines:
                County[i]=lines
       
                
                
    i +=1
    
                
                

In [7]:
df['bed'] = bed
df['bath'] = bath
df['price_sq'] = price_sq_feet
df['sqft'] = sqft
df['property_type'] = source
df['House_Style']=House_Style
df['year built'] = year_b
df['county'] = County

In [8]:
df

,price,link,city,sold_data,bed,bath,price_sq,sqft,property_type,House_Style,year built,county
0,"$465,000",/realestateandhomes-detail/4983-S-2000-W_Rexbu...,Rexburg_ID,"[{'date': '06/03/2022', 'sold_price': '$465,00...",Bedrooms: 3,Total Bathrooms: 1 / 2,\n\nPrice/Sq Ft\n$159\n,"2,280 sq ft",Source Property Type: Single Family,House Style: 1 Story,\n\nBuilt\n1960\n,County: Madison
1,"$499,000",/realestateandhomes-detail/3029-N-Salem-Rd_Rex...,Rexburg_ID,"[{'date': '06/01/2022', 'sold_price': '$499,00...",Bedrooms: 3,Total Bathrooms: 3 / 0,\n\nPrice/Sq Ft\n$206\n,"2,830 sq ft",Source Property Type: Single Family,House Style: 1 Story,\n\nBuilt\n1977\n,County: Madison
2,"$253,000",/realestateandhomes-detail/271-W-1st-N_Rexburg...,Rexburg_ID,"[{'date': '06/25/2022', 'sold_price': '$253,00...",Bedrooms: 3,Total Bathrooms: 1 / 0,\n\nPrice/Sq Ft\n$262\n,"1,458 sq ft",Source Property Type: Single Family,House Style: 1 Story,\n\nBuilt\n1960\n,County: Madison
3,"$595,000",/realestateandhomes-detail/253-E-5th-S_Rexburg...,Rexburg_ID,"[{'date': '06/06/2022', 'sold_price': '$595,00...",Bedrooms: 5,Total Bathrooms: 3 / 1,\n\nPrice/Sq Ft\n$170\n,"3,222 sq ft",Source Property Type: Single Family,House Style: 1 Story,\n\nBuilt\n1998\n,County: Madison
4,"$375,000",/realestateandhomes-detail/6344-S-3100-W_Rexbu...,Rexburg_ID,"[{'date': '06/23/2022', 'sold_price': '$375,00...",Bedrooms: 3,Total Bathrooms: 2 / 0,\n\nPrice/Sq Ft\n$178\n,Lot (sq ft),Source Property Type: Single Family,House Style: 1 Story,\n\nBuilt\n1978\n,County: Madison
...,...,...,...,...,...,...,...,...,...,...,...,...
163,"$629,990",/realestateandhomes-detail/8333-S-Snow-Bird-Av...,Boise_ID,"[{'date': '06/23/2022', 'sold_price': '$629,99...",Bedrooms: 4,Total Bathrooms: 3,\n\nPrice/Sq Ft\n$249\n,"2,153 sq ft",Source Property Type: Residential,0,\n\nBuilt\n2022\n,County: Ada
164,"$725,000",/realestateandhomes-detail/9641-W-Sultana-Dr_G...,Boise_ID,"[{'date': '06/24/2022', 'sold_price': '$725,00...",Bedrooms: 2,Total Bathrooms: 2,\n\nPrice/Sq Ft\n$404\n,"1,997 sq ft",Source Property Type: Residential,Roof: Architectural Style,\n\nBuilt\n2008\n,County: Ada
165,"$925,000",/realestateandhomes-detail/2926-S-Barnside-Way...,Boise_ID,"[{'date': '06/23/2022', 'sold_price': '$925,00...",Bedrooms: 5,Total Bathrooms: 3,\n\nPrice/Sq Ft\n$318\n,"2,681 sq ft",Source Property Type: Residential,0,\n\nBuilt\n2015\n,County: Ada
166,"$619,990",/realestateandhomes-detail/8351-S-Snow-Bird-Av...,Boise_ID,"[{'date': '06/23/2022', 'sold_price': '$619,99...",Bedrooms: 4,Total Bathrooms: 3,\n\nPrice/Sq Ft\n$288\n,"2,530 sq ft",Source Property Type: Residential,0,\n\nBuilt\n2022\n,County: Ada


In [9]:
price_change=[]
LP=0

for key in df['sold_data']:
    try:
        if key != []:
            if len(key) < 2:
                recent_price=key[0]
                r_price_final=Decimal(sub(r'[^\d.]', '',recent_price['sold_price']))
                list_price_final=Decimal(sub(r'[^\d.]', '',df['price'].iloc[LP]))
                final_price = ((list_price_final - r_price_final)/r_price_final)*100
                price_change.append(final_price)
                LP+=1
            if len(key) > 1:
                recent_price=key[0]
                r_price_final=Decimal(sub(r'[^\d.]', '',recent_price['sold_price']))
                early_price=key[-1]
                e_price_final=Decimal(sub(r'[^\d.]', '',early_price['sold_price']))
                final_price = ((r_price_final - e_price_final)/e_price_final)*100
                price_change.append(round(final_price,2))
                LP+=1
        else:
            price_change.append(0)
            LP+=1
    except Exception as e:
        price_change.append(0)
        LP+=1

    
df['price_change'] = price_change

In [10]:
df.to_csv(str(city)+".csv")


for i in summary:
    for j in i:
        test=j.find_all('li')
        print(test)
        for ti in test:
            print(ti.get_text())
            input("")


for sp in span_total:
    for ul in sp:
        for li in ul:
            lines=li.get_text()  
            print(lines)
            
          